#### Model: 

\begin{eqnarray}
\text{Latent:} \quad X_t & = & A X_{t-1}  + \nu_t, 
\\
\text{Observed:} \quad Y_t & = & C X_t + B Z_t + \omega_t, \quad Z_t ~ \text{are covariates}
\\
\nu_t & \sim & \text{N}(0, Q ),
\\
\omega_t & \sim & \text{N}(0, R).
\end{eqnarray}

In [1]:
from __future__ import division
%matplotlib inline
from pykalman import KalmanFilter
import numpy as np, numpy.random as npr, matplotlib.pyplot as plt, copy, multiprocessing as mp, csv, pandas
from scipy.stats import *
from pylab import plot, show, legend
from tqdm import trange
from ozone_functions import *

In [2]:
data = pandas.read_csv("data.csv").values
data = data[:,1::]

In [3]:
T = np.shape(data)[0]
Y = data[:,0:3]
Z = data[:,3::]

obs_dim = np.shape(Y)[-1]
lat_dim = 1
cov_dim = np.shape(Z)[-1]

In [4]:
A = npr.randn(lat_dim,lat_dim)
C = npr.randn(obs_dim,lat_dim)
sigmax2 = 1
sigmay2 = 1

Q = sigmax2*np.eye(lat_dim)
R = sigmay2*np.eye(obs_dim)

B = npr.randn(obs_dim,cov_dim)
b = np.transpose(np.matmul(B,Z.transpose()))

In [5]:
mu0 = np.zeros(lat_dim)
Sigma0 = np.eye(lat_dim)

In [6]:
n_mcmc = 10_000
scale = np.asarray([3e-2,3e-2])
scale_A = 3e-2*np.ones(np.shape(A))
scale_C = 3e-2*np.ones(np.shape(C))
scale_B = 3e-2*np.ones(np.shape(B))

In [7]:
K = 10
tstarts = int(T/K)*np.arange(K).astype(int)
Ychunks = [Y[tstart:(int(T/K)+tstart)] for tstart in tstarts]
Zchunks = [Z[tstart:(int(T/K)+tstart)] for tstart in tstarts]

def f(y, z) :
    return adaptive_MH(y, Z, A, C, B, sigmax2, sigmay2, mu0, Sigma0, n_mcmc, 
                       scale, scale_A, scale_C, scale_B, power=K, adapt=False, kappa=3e-1)

In [8]:
pool = mp.Pool(np.minimum(K,10))
results = pool.starmap(f, zip(Ychunks,Zchunks))
pool.close()

10.0 % run in 7.8 mins; acceptance rate = 0.28
10.0 % run in 8.4 mins; acceptance rate = 0.2
10.0 % run in 8.7 mins; acceptance rate = 0.24
10.0 % run in 8.7 mins; acceptance rate = 0.25
10.0 % run in 8.7 mins; acceptance rate = 0.22
10.0 % run in 8.8 mins; acceptance rate = 0.24
10.0 % run in 8.9 mins; acceptance rate = 0.28
10.0 % run in 9.0 mins; acceptance rate = 0.22
10.0 % run in 9.0 mins; acceptance rate = 0.23
10.0 % run in 9.0 mins; acceptance rate = 0.17
20.0 % run in 16.1 mins; acceptance rate = 0.15
20.0 % run in 17.1 mins; acceptance rate = 0.12
20.0 % run in 17.1 mins; acceptance rate = 0.1
20.0 % run in 17.3 mins; acceptance rate = 0.13
20.0 % run in 17.3 mins; acceptance rate = 0.12
20.0 % run in 17.4 mins; acceptance rate = 0.08
20.0 % run in 17.5 mins; acceptance rate = 0.13
20.0 % run in 17.6 mins; acceptance rate = 0.12
20.0 % run in 17.9 mins; acceptance rate = 0.12
20.0 % run in 18.0 mins; acceptance rate = 0.14
30.0 % run in 25.1 mins; acceptance rate = 0.1
30.0 